<a href="https://colab.research.google.com/github/MarciaFG/scimobility/blob/main/transformation_index_for_2007_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Transformative Flows Project (2007-2022)**
**Author:** Marcia R. Ferreira (Complexity Science Hub Vienna & TU Wien)
- **Inputs:** 

1.   CWTS SQL Server [dimensions_2022jun]:


      *   Exported File:
      *   Exported File:


2.   CWTS Publication-level classification system: Meso-fields level [dimensions_2022jun_classification]
3.   Dimension reduction-based clustering: Laplacian matrix contructed from meso-field level topic matrix and second eigenvector of the matrix
4.   Dimensions database on BigQuery


- **Outputs:**

### Initialization and drivers

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime → "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Thu Apr 13 14:43:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
#!pip install psutil
#!pip install humanize
#!pip install pynput
#pip install plotly==5.4.0
!pip install patool

# main libraries
import psutil
import humanize
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import requests
import torch
import nltk
import GPUtil as GPU

# plotting
import plotly.graph_objs as go
import plotly.io as pio
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

from google.cloud import bigquery
from google.colab import files
%load_ext google.colab.data_table
%load_ext google.cloud.bigquery

from google.colab import drive
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7408 sha256=04336840c20cf1011bfcfa5b5630ede460255ac186f5c976ef1125f67aeb1f7c
  Stored in directory: /root/.cache/pip/wheels/2b/b5/24/fbb56595c286984f7315ee31821d6121e1b9828436021a88b3
Successfully built gputil
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 3.4 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
# Provide your credentials to the runtime
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

# declare your project 
project_id = "cshdimensionstest"

### Data imports

In [ ]:
""" NOT RUN
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id = '1wCFzWEAwBqH47qGQG1_-G6wPgrrs03A6'
print(id) # Verify that you have everything

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('second_eigenvector_clustering.csv')  
clusters = pd.read_csv('second_eigenvector_clustering.csv', sep=",", index_col=0) # Dataset is now stored in a Pandas Dataframe

print(clusters.head(10))
print("The data types are as follows:\n", clusters.dtypes)
print("The type of object is:\n", type(clusters))
"""

In [ ]:
""" NOT RUN
# unzip the files exported from SQL Server
#!unzip "/content/drive/My Drive/TRANSFORMATION/data_export.zip"
#!unzip "/content/drive/My Drive/TRANSFORMATION/data_export.zip" > /dev/null
"""

In [ ]:
import patoolib
import tempfile
import os

# Path of the zip file in Google Drive
zip_path = "/content/drive/My Drive/TRANSFORMATION/data_export.zip"

# Name of the CSV file(s) inside the zip
csv_file_names = [  "spectral_meso_clusters.csv"
                  , "for_division_labels.csv"
                  , "grid_ranks.csv"
                  , "trajectories_au_fourfive_skill.csv"
                  , "trajectories_au_morethanfive_skill.csv"
                  , "trajectories_au_single_skill.csv"
                  , "trajectories_au_twothree_skill.csv"]

# Separator character to use in the CSV files
separator = ";"

# Extract the zip file to a temporary directory
with tempfile.TemporaryDirectory() as tmpdir:
    patoolib.extract_archive(zip_path, outdir=tmpdir)
    
    # Load each CSV file into its own dataframe
    dfs = []
    for csv_file_name in csv_file_names:
        csv_file_path = os.path.join(tmpdir, csv_file_name)
        try:
            df = pd.read_csv(csv_file_path, sep=separator, encoding='utf-8', header= None, decimal=".")
            dfs.append(df)
        except pd.errors.ParserError:
            print(f"Error loading {csv_file_name}: Skipping...")

# Print the first few rows of each dataframe
for i, df in enumerate(dfs):
    print(f"Dataframe {i}:")
    print(df.head(2))
print("###########################################")

# extract the datasets and store them into a pandas dataframe
spectral_meso_clusters = dfs[0]
for_division_labels = dfs[1]
grid_ranks = dfs[2]
trajectories_au_fourfive_skill = dfs[3]
trajectories_au_morethanfive_skill = dfs[4]
trajectories_au_single_skill = dfs[5]
trajectories_au_twothree_skill = dfs[6]

print(type(for_division_labels))
print("###########################################")

In [ ]:
# extract the datasets and store them into a pandas dataframe
spectral_meso_clusters = dfs[0]
for_division_labels = dfs[1]
grid_ranks = dfs[2]
trajectories_au_fourfive_skill = dfs[3]
trajectories_au_morethanfive_skill = dfs[4]
trajectories_au_single_skill = dfs[5]
trajectories_au_twothree_skill = dfs[6]

print(type(for_division_labels))

## Preprocessing

In [2]:
# Use the first row as the header
spectral_meso_clusters.columns = spectral_meso_clusters.iloc[0]
for_division_labels.columns = for_division_labels.iloc[0]
grid_ranks.columns = grid_ranks.iloc[0]

# Remove the first row (which is now the header)
spectral_meso_clusters = spectral_meso_clusters[1:]
for_division_labels = for_division_labels[1:]
grid_ranks = grid_ranks[1:]

print(spectral_meso_clusters.head())
print(for_division_labels.head())
print(grid_ranks.head())
print("###########################################")

def convert_to_float(val):
    if isinstance(val, str) and val.replace('.', '', 1).isdigit():
        return float(val.replace(',', '.'))
    return val

# Apply the function to all elements of the dataframe
grid_ranks = grid_ranks.applymap(convert_to_float)
spectral_meso_clusters = spectral_meso_clusters.applymap(convert_to_float)

print(grid_ranks.dtypes)
print(spectral_meso_clusters.dtypes)
print("###########################################")


from pandas.core.dtypes.dtypes import dtypes
from numpy.core.multiarray import dtype

headers = ['researcher_id', 'grid_id', 'start', 'end', 'Lenght', 'for_division_id', 'meso_field', 'spectral_cluster_id', 'concatenated_fields', 'year', 'n_pubs']

# set the new column names using the list
trajectories_au_morethanfive_skill.columns = headers
trajectories_au_fourfive_skill.columns = headers
trajectories_au_single_skill.columns = headers
trajectories_au_twothree_skill.columns = headers

# print the updated column names
print(trajectories_au_morethanfive_skill.columns)
print(trajectories_au_morethanfive_skill.dtypes)
print("###########################################")

print(trajectories_au_morethanfive_skill.describe())

NameError: ignored

In [1]:
def calculate_org_sequence(df):
    # select the desired columns and drop duplicates
    df = df[['researcher_id', 'grid_id', 'start', 'end']].drop_duplicates().reset_index(drop=True)
    
    # calculate the org_sequence using rank()
    df['org_sequence'] = df.groupby('researcher_id')['start'].rank(method='dense')
    
    return df

# calculate org_sequence for each dataframe
sq_1_skill_df = calculate_org_sequence(trajectories_au_single_skill)
sq_2_3_skill_df = calculate_org_sequence(trajectories_au_twothree_skill)
sq_4_5_skill_df = calculate_org_sequence(trajectories_au_fourfive_skill)
sq_5_or_more_skill_df = calculate_org_sequence(trajectories_au_morethanfive_skill)

# merge the two dataframes on researcher_id and grid_id
sq_1_skill_df = pd.merge(trajectories_au_single_skill, sq_1_skill_df, on=['researcher_id', 'grid_id'], how='left')
sq_1_skill_df = sq_1_skill_df.loc[:, ~sq_1_skill_df.columns.str.endswith('_y')]
sq_1_skill_df = sq_1_skill_df.rename(columns=lambda x: x[:-2] if x.endswith('_x') else x)
sq_1_skill_df['concatenated_2'] =  sq_1_skill_df['for_division_id'].astype(str) + ' - ' + sq_1_skill_df['spectral_cluster_id'].astype(str)
sq_1_skill_df = sq_1_skill_df[['researcher_id', 'grid_id', 'concatenated_2', 'org_sequence', 'n_pubs']].drop_duplicates().reset_index(drop=True)
sq_1_skill_df = sq_1_skill_df.groupby(['researcher_id', 'grid_id', 'concatenated_2', 'org_sequence']).sum().reset_index()

sq_2_3_skill_df = pd.merge(trajectories_au_twothree_skill, sq_2_3_skill_df, on=['researcher_id', 'grid_id'], how='left')
sq_2_3_skill_df = sq_2_3_skill_df.loc[:, ~sq_2_3_skill_df.columns.str.endswith('_y')]
sq_2_3_skill_df = sq_2_3_skill_df.rename(columns=lambda x: x[:-2] if x.endswith('_x') else x)
sq_2_3_skill_df['concatenated_2'] =  sq_2_3_skill_df['for_division_id'].astype(str) + ' - ' + sq_2_3_skill_df['spectral_cluster_id'].astype(str)
sq_2_3_skill_df = sq_2_3_skill_df[['researcher_id', 'grid_id', 'concatenated_2', 'org_sequence', 'n_pubs']].drop_duplicates().reset_index(drop=True)
sq_2_3_skill_df = sq_2_3_skill_df.groupby(['researcher_id', 'grid_id', 'concatenated_2', 'org_sequence']).sum().reset_index()

sq_4_5_skill_df = pd.merge(trajectories_au_fourfive_skill, sq_4_5_skill_df, on=['researcher_id', 'grid_id'], how='left')
sq_4_5_skill_df = sq_4_5_skill_df.loc[:, ~sq_4_5_skill_df.columns.str.endswith('_y')]
sq_4_5_skill_df = sq_4_5_skill_df.rename(columns=lambda x: x[:-2] if x.endswith('_x') else x)
sq_4_5_skill_df['concatenated_2'] =  sq_4_5_skill_df['for_division_id'].astype(str) + ' - ' + sq_4_5_skill_df['spectral_cluster_id'].astype(str)
sq_4_5_skill_df = sq_4_5_skill_df[['researcher_id', 'grid_id', 'concatenated_2', 'org_sequence', 'n_pubs']].drop_duplicates().reset_index(drop=True)
sq_4_5_skill_df = sq_4_5_skill_df.groupby(['researcher_id', 'grid_id', 'concatenated_2', 'org_sequence']).sum().reset_index()

sq_5_or_more_skill_df = pd.merge(trajectories_au_morethanfive_skill, sq_5_or_more_skill_df, on=['researcher_id', 'grid_id'], how='left')
sq_5_or_more_skill_df = sq_5_or_more_skill_df.loc[:, ~sq_5_or_more_skill_df.columns.str.endswith('_y')]
sq_5_or_more_skill_df = sq_5_or_more_skill_df.rename(columns=lambda x: x[:-2] if x.endswith('_x') else x)
sq_5_or_more_skill_df['concatenated_2'] =  sq_5_or_more_skill_df['for_division_id'].astype(str) + ' - ' + sq_5_or_more_skill_df['spectral_cluster_id'].astype(str)
sq_5_or_more_skill_df = sq_5_or_more_skill_df[['researcher_id', 'grid_id', 'concatenated_2', 'org_sequence', 'n_pubs']].drop_duplicates().reset_index(drop=True)
sq_5_or_more_skill_df = sq_5_or_more_skill_df.groupby(['researcher_id', 'grid_id', 'concatenated_2', 'org_sequence']).sum().reset_index()

print(sq_1_skill_df.head())
print(len(sq_1_skill_df))
print(len(trajectories_au_single_skill))

#print(sq_1_skill_df.head(10))
# select all rows that have org_sequence > 1
#sq_1_skill_df_filtered = sq_1_skill_df[sq_1_skill_df['org_sequence'] > 1]
#print(sq_1_skill_df_filtered.head(10))
# select all rows that have researcher_id = 'ur.01000012260.80'
#sq_1_skill_df_filtered_au = sq_1_skill_df[sq_1_skill_df['researcher_id'] == 'ur.01000012260.80']
#print(sq_1_skill_df_filtered_au.head(10))

NameError: ignored

In [ ]:
def process_dataframe(df, org_seq_df):
    # merge the dataframes on researcher_id and grid_id
    merged_df = pd.merge(df, org_seq_df, on=['researcher_id', 'grid_id'], how='left')
    merged_df = merged_df.loc[:, ~merged_df.columns.str.endswith('_y')]
    merged_df = merged_df.rename(columns=lambda x: x[:-2] if x.endswith('_x') else x)

    # concatenate two columns
    merged_df['concatenated_2'] = merged_df['for_division_id'].astype(str) + ' - ' + merged_df['spectral_cluster_id'].astype(str)

    # select and aggregate columns
    selected_cols = ['researcher_id', 'grid_id', 'for_division_id', 'concatenated_2', 'org_sequence', 'n_pubs']
    selected_df = merged_df[selected_cols].drop_duplicates().reset_index(drop=True)
    aggregated_df = selected_df.groupby(['researcher_id', 'grid_id','for_division_id', 'concatenated_2','org_sequence']).sum().reset_index()

    return aggregated_df

sq_1_skill_df = process_dataframe(trajectories_au_single_skill, calculate_org_sequence(trajectories_au_single_skill))
sq_2_3_skill_df = process_dataframe(trajectories_au_twothree_skill, calculate_org_sequence(trajectories_au_twothree_skill))
sq_4_5_skill_df = process_dataframe(trajectories_au_fourfive_skill, calculate_org_sequence(trajectories_au_fourfive_skill))
sq_5_or_more_skill_df = process_dataframe(trajectories_au_morethanfive_skill, calculate_org_sequence(trajectories_au_morethanfive_skill))

print(sq_1_skill_df.head())
print(len(sq_1_skill_df))
print(len(trajectories_au_single_skill))

**Looks good!**

## Cosine similarity scores

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def calculate_cosine_similarity(df):
    # Sort the dataframe by researcher_id, org_sequence, and year_published
    df = df.sort_values(['researcher_id', 'org_sequence'])
    
    # Get unique combinations of researcher_id, org_sequence, grid_id, and for_division_id
    unique_combinations = df[['researcher_id', 'org_sequence', 'grid_id', 'for_division_id']].drop_duplicates()
    
    # Create an empty dataframe to store the results
    results = pd.DataFrame(columns=['researcher_id', 'prev_org_sequence', 'next_org_sequence', 'prev_grid_id', 'next_grid_id', 'prev_for_division_id','next_for_division_id', 'cosine_similarity'])
    
    # Iterate over each unique combination of researcher_id, org_sequence, grid_id, and for_division_id
    for idx, row in unique_combinations.iterrows():
        researcher_id = row['researcher_id']
        org_sequence = row['org_sequence']
        grid_id = row['grid_id']
        for_division_id = row['for_division_id']
        
        # Filter the dataframe to include only rows with the same researcher_id, org_sequence, grid_id, and for_division_id
        filtered_df = df[(df['researcher_id'] == researcher_id) & (df['org_sequence'] == org_sequence) & (df['grid_id'] == grid_id) & (df['for_division_id'] == for_division_id)]
        
        # Create a pivot table with concatenated_2 as columns, and n_pubs as values
        pivot_table = filtered_df.pivot_table(values='n_pubs', index='org_sequence', columns='concatenated_2', aggfunc=np.sum, fill_value=0)
        
        # Use dynamic programming to calculate the cosine similarity between each adjacent pair of rows in the pivot table
        similarity_scores = []
        for i in range(len(pivot_table) - 1):
            prev_row = pivot_table.iloc[[i]]
            next_row = pivot_table.iloc[[i+1]]
            cosine_sim = cosine_similarity(prev_row, next_row)[0][0]
            similarity_scores.append(cosine_sim)
        
        # Append the results to the results dataframe
        if len(similarity_scores) > 0:
            prev_org_sequence = org_sequence - 1
            next_org_sequence = org_sequence + 1 if org_sequence < filtered_df['org_sequence'].max() else None
            prev_grid_id = filtered_df[filtered_df['org_sequence'] == prev_org_sequence]['grid_id'].values[0] if prev_org_sequence in filtered_df['org_sequence'].values else None
            next_grid_id = filtered_df[filtered_df['org_sequence'] == next_org_sequence]['grid_id'].values[0] if next_org_sequence is not None and next_org_sequence in filtered_df['org_sequence'].values else None
            result_row = {'researcher_id': researcher_id, 'prev_org_sequence': prev_org_sequence, 'next_org_sequence': next_org_sequence,
                          'prev_grid_id': prev_grid_id, 'next_grid_id': next_grid_id, 'prev_for_division_id': prev_for_division_id, 'next_for_division_id': next_for_division_id,
                          'cosine_similarity': similarity_scores}
            results = results.append(result_row, ignore_index=True)
            
    # Return the results dataframe
    return results
calculate_cosine_similarity(sq_1_skill_df)
